# **STAT 306 Group Project: Predicting Sleep Efficiency**

>##### By: Pahul Brar, Diana Liang, Peng Xie, Jonathan Tuazon

## 1. Introduction

Sleep is an essential aspect of human life, and its quality has a significant impact on our overall health and well-being. However, many people struggle with getting good quality sleep, which can lead to a variety of negative health outcomes. To address this issue, we have undertaken a project to predict sleep efficiency using various data sources and learning techniques.

In this project, we aim to develop a model that can accurately predict the quality of sleep based on various factors, such as the duration of sleep, caffeine consumption, deep sleep percentage, and light sleep percentage. By predicting sleep efficiency, we can identify potential causes of poor sleep and provide recommendations to improve it.

To achieve this goal, we will be collecting data from the Kaggle and analyze the data to identify patterns and trends that are associated with sleep efficiency. Finally, we will develop a model that can predict sleep efficiency based on these factors.

The potential benefits of this project are important, as improving sleep efficiency can have a positive impact on many areas of our lives. We hope that our research will lead to a better understanding of sleep and its relationship with various lifestyle factors, and ultimately contribute to the development of more effective interventions to improve sleep quality.

**Research Question:** "Research question goes here"

(TO EDIT AND EXPAND) Which set of factors play a significant role in accurately predicting sleep efficiency?

"Motivation goes here"

(TO EDIT AND EXPAND) In this project we aim to determine, using data collected by a team of researchers from the University of Oxfordshire, what factors play a significant role in accurately predicting sleep efficiency. We attempt to use either a linear model or a quadratic model, whichever is more suitable and gives us better results to approach this. The motivation behind this is that sleep is a crucial element many college students are forced to ignore and rather just rely on caffeine consumption to function. We want to determine if any relationship exists between items like Coffee, Cigarettes, Alcohol, or sleep patterns and sleep efficiency. It has been repeatedly proved that sleep deprivation leads to a reduction in alertness, attention, and other basic cognitive processes (Garcia et al.).  “Sleep deprivation is more detrimental to cognitive functions that depend more on mental effort and cognitive capacity,” such as reaction time (Kusztor et al.). Sleep is an important aspect for humans, and thus we want to explore how to maximize its efficiency and determine if any factors are more detrimental to it.

#### Brief Description of the Variables in the Dataset

"Brief Description goes here"

(TO EDIT AND EXPAND) The dataset used in this project is sourced from [Kaggle](https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency). The sleep efficiency data was collected in a study conducted by a team of researchers from the University of Oxfordshire. Over the course of several months, the team collected data using self-reported surveys, actigraphy, and polysomnography from participants they recruited from the local community in the UK. The source did not provide any further information.

The following include all the variables measured in the dataset:
- Age (years) 
    - Age of the individual
- Gender (male or female)
    - Categorical: Subject is male or female
- Bedtime (timestamp: YYYY-MM-DD HH:MM: SS) 
    - Time subject goes to bed
- Wakeup time (timestamp: YYYY-MM-DD HH:MM: SS)
    - Time subject wakes up
- Sleep duration (hours)
    - The total amount of time spent sleeping
- Sleep efficiency
    - proportion of time in bed spent asleep
- REM sleep percentage
    - percentage of sleep duration spent in REM sleep
- Deep sleep percentage
    - percentage of sleep duration spent in Deep sleep
- Light sleep percentage
    - percentage of sleep duration spent in Light sleep
- Awakenings
    - number of times the subject woke up during the night
- Caffeine Consumption (mg)
    - Amount of caffeine consumed in the 24 hours prior to bedtime
- Alcohol Consumption (oz)
    - Amount of alcohol consumed in the 24 hours prior to bedtime
- Smoker
    - Categorical: subject smokes or does not smoke
- Exercise frequency
    - number of times the subject exercises each week

## 2. Analysis

#### Loading Packages and Cleaning the Dataset

In [39]:
library(tidyverse)
library(leaps)

In [40]:
data <- read.csv("Sleep_Efficiency.csv") |>
    na.omit(data) |>
    select(-c(Bedtime,Wakeup.time,ID))
data$Smoking.status <- ifelse(data$Smoking.status=="Yes",1,0) |>
    as.factor()
data$Gender <- ifelse(data$Gender=="Male",1,0) |>
    as.factor()
head(data)

n <- nrow(data)

,Age,Gender,Sleep.duration,Sleep.efficiency,REM.sleep.percentage,Deep.sleep.percentage,Light.sleep.percentage,Awakenings,Caffeine.consumption,Alcohol.consumption,Smoking.status,Exercise.frequency
,<int>,<fct>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,65,0,6,0.88,18,70,12,0,0,0,1,3
2,69,1,7,0.66,19,28,53,3,0,3,1,3
3,40,0,8,0.89,20,70,10,1,0,0,0,3
4,40,0,6,0.51,23,25,52,3,50,5,1,1
5,57,1,8,0.76,27,55,18,3,0,3,0,3
7,27,0,6,0.54,28,25,47,2,50,0,1,1


#### Summary of Statistics

In [41]:
summary(data)

      Age        Gender  Sleep.duration   Sleep.efficiency REM.sleep.percentage
 Min.   : 9.00   0:194   Min.   : 5.000   Min.   :0.5000   Min.   :15.00       
 1st Qu.:29.00   1:194   1st Qu.: 7.000   1st Qu.:0.7000   1st Qu.:20.00       
 Median :41.00           Median : 7.500   Median :0.8200   Median :22.00       
 Mean   :40.83           Mean   : 7.451   Mean   :0.7893   Mean   :22.68       
 3rd Qu.:52.00           3rd Qu.: 8.000   3rd Qu.:0.9000   3rd Qu.:25.00       
 Max.   :69.00           Max.   :10.000   Max.   :0.9900   Max.   :30.00       
 Deep.sleep.percentage Light.sleep.percentage   Awakenings   
 Min.   :18.00         Min.   : 7.0           Min.   :0.000  
 1st Qu.:51.00         1st Qu.:15.0           1st Qu.:1.000  
 Median :58.00         Median :18.0           Median :1.000  
 Mean   :52.82         Mean   :24.5           Mean   :1.619  
 3rd Qu.:63.00         3rd Qu.:24.0           3rd Qu.:3.000  
 Max.   :75.00         Max.   :63.0           Max.   :4.000  
 Caffe

#### Correlation Matrix

In [42]:
## Code for the correlation matrix goes here

#find correlation between numeric data
data_cor_numeric <- cor(data[, sapply(data, is.numeric)])
round(data_cor_numeric, 2)

#find correlation matrix with significance levels
library(Hmisc)
data_cor <- rcorr(as.matrix(data))
data_cor

,Age,Sleep.duration,Sleep.efficiency,REM.sleep.percentage,Deep.sleep.percentage,Light.sleep.percentage,Awakenings,Caffeine.consumption,Alcohol.consumption,Exercise.frequency
Age,1.00,-0.07,0.12,0.02,0.06,-0.06,0.00,-0.17,0.07,0.07
Sleep.duration,-0.07,1.00,-0.02,-0.02,-0.04,0.04,-0.01,-0.03,-0.05,-0.05
Sleep.efficiency,0.12,-0.02,1.00,0.06,0.79,-0.82,-0.57,0.07,-0.40,0.27
REM.sleep.percentage,0.02,-0.02,0.06,1.00,-0.19,-0.04,-0.02,0.11,-0.04,0.04
Deep.sleep.percentage,0.06,-0.04,0.79,-0.19,1.00,-0.98,-0.33,-0.02,-0.37,0.17
Light.sleep.percentage,-0.06,0.04,-0.82,-0.04,-0.98,1.00,0.34,0.00,0.39,-0.18
Awakenings,0.00,-0.01,-0.57,-0.02,-0.33,0.34,1.00,-0.11,0.21,-0.23
Caffeine.consumption,-0.17,-0.03,0.07,0.11,-0.02,0.00,-0.11,1.00,-0.10,-0.08
Alcohol.consumption,0.07,-0.05,-0.40,-0.04,-0.37,0.39,0.21,-0.10,1.00,0.00
Exercise.frequency,0.07,-0.05,0.27,0.04,0.17,-0.18,-0.23,-0.08,0.00,1.00


                         Age Gender Sleep.duration Sleep.efficiency
Age                     1.00   0.27          -0.07             0.12
Gender                  0.27   1.00          -0.05            -0.01
Sleep.duration         -0.07  -0.05           1.00            -0.02
Sleep.efficiency        0.12  -0.01          -0.02             1.00
REM.sleep.percentage    0.02  -0.11          -0.02             0.06
Deep.sleep.percentage   0.06   0.05          -0.04             0.79
Light.sleep.percentage -0.06  -0.02           0.04            -0.82
Awakenings              0.00   0.08          -0.01            -0.57
Caffeine.consumption   -0.17  -0.28          -0.03             0.07
Alcohol.consumption     0.07   0.05          -0.05            -0.40
Smoking.status          0.02   0.15           0.00            -0.29
Exercise.frequency      0.07   0.22          -0.05             0.27
                       REM.sleep.percentage Deep.sleep.percentage
Age                                    0.02       

"Summarize the findings from the correlation matrix"

#### Distribution of the Response Variable (Sleep.efficiency)

In [43]:
## Code for the plot goes here

"Summarize the plot (if needed)"

#### Model Selection - Full Model

In [44]:
set.seed(2023)

step_data <- data |>
    select(-c(Smoking.status,Gender))
step_data$ID <- 1:n
step_data_train <- sample_n(step_data,size=0.5*n,replace=FALSE)
step_data_test <- anti_join(step_data,step_data_train,by="ID")

step_data_train <- step_data_train |> 
    select(-ID)
step_data_test <- step_data_test |> 
    select(-ID)

rmse <- function(u,v) {sqrt(mean((u-v)^2))}
head(step_data_train)

,Age,Sleep.duration,Sleep.efficiency,REM.sleep.percentage,Deep.sleep.percentage,Light.sleep.percentage,Awakenings,Caffeine.consumption,Alcohol.consumption,Exercise.frequency
,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,37,7.0,0.92,25,55,20,1,50,1,3
2,44,7.0,0.66,20,35,45,4,0,3,3
3,25,7.5,0.90,28,55,17,1,50,0,1
4,29,7.5,0.50,22,28,50,1,75,1,2
5,29,7.0,0.71,22,57,21,1,25,0,2
6,58,7.0,0.68,18,35,47,1,0,4,3


In [45]:
full_model <- lm(Sleep.efficiency~.,data=step_data_train)
pre_full_model <- predict(full_model,step_data_test)

full_model_RMSE <- tibble(Model="Full Model",
                          RMSE=rmse(step_data_train$Sleep.efficiency,pre_full_model))
full_model_RMSE

Warning message in predict.lm(full_model, step_data_test):
“prediction from a rank-deficient fit may be misleading”


Model,RMSE
<chr>,<dbl>
Full Model,0.1780283


#### Model Selection - Forward and Backward Selection

In [46]:
forward_select <- summary(regsubsets(x=Sleep.efficiency~.,
                                    nvmax=9,
                                    data=step_data_train,
                                    method="forward"))
forward_cp <- tibble(var=1:8,cp=forward_select$cp) |>
    arrange(cp)
forward_select
forward_cp

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
“1  linear dependencies found”


Reordering variables and trying again:


Warning message in rval$lopt[] <- rval$vorder[rval$lopt]:
“number of items to replace is not a multiple of replacement length”


Subset selection object
Call: regsubsets.formula(x = Sleep.efficiency ~ ., nvmax = 9, data = step_data_train, 
    method = "forward")
9 Variables  (and intercept)
                       Forced in Forced out
Age                        FALSE      FALSE
Sleep.duration             FALSE      FALSE
REM.sleep.percentage       FALSE      FALSE
Deep.sleep.percentage      FALSE      FALSE
Awakenings                 FALSE      FALSE
Caffeine.consumption       FALSE      FALSE
Alcohol.consumption        FALSE      FALSE
Exercise.frequency         FALSE      FALSE
Light.sleep.percentage     FALSE      FALSE
1 subsets of each size up to 8
Selection Algorithm: forward
         Age Sleep.duration REM.sleep.percentage Deep.sleep.percentage
1  ( 1 ) " " " "            " "                  " "                  
2  ( 1 ) " " " "            " "                  " "                  
3  ( 1 ) "*" " "            " "                  " "                  
4  ( 1 ) "*" " "            " "                  " "

var,cp
<int>,<dbl>
4,2.740637
5,3.529309
3,3.853241
6,4.713344
2,5.647597
7,6.089748
8,8.000000
1,87.558009


In [47]:
forward_model <- lm(Sleep.efficiency~Light.sleep.percentage+Awakenings+Age+Exercise.frequency,data=step_data_train)
pre_forward_model <- predict(forward_model,step_data_test)
forward_model_RMSE <- tibble(Model="Forward Model",
                             RMSE=rmse(step_data_train$Sleep.efficiency,pre_forward_model))
forward_model_RMSE

Model,RMSE
<chr>,<dbl>
Forward Model,0.1787038


In [48]:
backward_select <- summary(regsubsets(x=Sleep.efficiency~.,
                                    nvmax=9,
                                    data=step_data_train,
                                    method="backward"))
backward_cp <- tibble(var=1:8,cp=backward_select$cp) |>
    arrange(cp)
backward_select
backward_cp

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
“1  linear dependencies found”


Reordering variables and trying again:


Warning message in rval$lopt[] <- rval$vorder[rval$lopt]:
“number of items to replace is not a multiple of replacement length”


Subset selection object
Call: regsubsets.formula(x = Sleep.efficiency ~ ., nvmax = 9, data = step_data_train, 
    method = "backward")
9 Variables  (and intercept)
                       Forced in Forced out
Age                        FALSE      FALSE
Sleep.duration             FALSE      FALSE
REM.sleep.percentage       FALSE      FALSE
Deep.sleep.percentage      FALSE      FALSE
Awakenings                 FALSE      FALSE
Caffeine.consumption       FALSE      FALSE
Alcohol.consumption        FALSE      FALSE
Exercise.frequency         FALSE      FALSE
Light.sleep.percentage     FALSE      FALSE
1 subsets of each size up to 8
Selection Algorithm: backward
         Age Sleep.duration REM.sleep.percentage Deep.sleep.percentage
1  ( 1 ) " " " "            " "                  "*"                  
2  ( 1 ) " " " "            " "                  "*"                  
3  ( 1 ) " " " "            "*"                  "*"                  
4  ( 1 ) "*" " "            "*"                  "

var,cp
<int>,<dbl>
5,3.529309
6,4.713344
4,4.812718
7,6.089748
3,6.685885
8,8.000000
2,32.272539
1,129.197743


In [49]:
backward_model <- lm(Sleep.efficiency~Deep.sleep.percentage+Awakenings+REM.sleep.percentage+Age+Exercise.frequency,data=step_data_train)
pre_backward_model <- predict(backward_model,step_data_test)
backward_model_RMSE <- tibble(Model="Backward Model",
                             RMSE=rmse(step_data_train$Sleep.efficiency,pre_backward_model))
backward_model_RMSE

Model,RMSE
<chr>,<dbl>
Backward Model,0.1782972


#### RMSE Comparison of the Models

In [56]:
rbind(full_model_RMSE,forward_model_RMSE,backward_model_RMSE) |>
arrange(RMSE)

Model,RMSE
<chr>,<dbl>
Full Model,0.1780283
Backward Model,0.1782972
Forward Model,0.1787038


"Describe/Compare the results here"

basically, we just use all of the variables for our linear and quadratic models. i'll add more detail to this explanation later.

#### Linear Models

In [50]:
# Code goes here

"Description of results go here"

#### Quadratic Models

In [51]:
# Code goes here

"Description of results go here"

#### Error Analysis (Extra if needed, Residual Plots and stuff like that ya know)

In [52]:
# Code goes here

"Description of plot goes here"

## 3. Conclusion

"Discuss results and findings here, talk about the model selection process, which one was the best, compare the R^2 of the linear and quadratic models, etc..."

#### Improvements

"Write improvements and limitiations of the project here"

#### References

García, A., Angel, J. D., Borrani, J., Ramirez, C., & Valdez, P. (2021). Sleep deprivation effects on basic cognitive processes: which components of attention, working memory, and executive functions are more susceptible to the lack of sleep?. *Sleep science (Sao Paulo, Brazil)*, 14(2), 107–118. <https://doi.org/10.5935/1984-0063.20200049> 
                            
Anikó Kusztor, Liisa Raud, Bjørn E Juel, André S Nilsen, Johan F Storm, Rene J Huster, Sleep deprivation differentially affects subcomponents of cognitive control, *Sleep*, Volume 42, Issue 4, April 2019, zsz016, <https://doi.org/10.1093/sleep/zsz016>
